# Build a RAG solution with Azure AI Search and Azure OpenAI

## Quickstart
Link: https://learn.microsoft.com/en-us/azure/search/search-get-started-rag

### Setup

In [ ]:
import json
import os
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from dotenv import load_dotenv
from openai import AzureOpenAI

In [ ]:
load_dotenv()

### Run

In [ ]:
openai_client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2024-02-01"
 )

In [ ]:
search_client = SearchClient(
    endpoint = os.getenv("AZURE_SEARCH_SERVICE"), 
    index_name="hotels-sample-index",
    credential=AzureKeyCredential(os.environ["AZURE_SEARCH_API_KEY"])
)

In [ ]:
GROUNDED_PROMPT="""
You are a friendly assistant that recommends hotels based on activities and amenities.
Answer the query using only the sources provided below in a friendly and concise bulleted manner.
Answer ONLY with the facts listed in the list of sources below.
If there isn't enough information below, say you don't know.
Do not generate answers that don't use the sources below.
Query: {query}
Sources:\n{sources}
"""

In [ ]:
query="Can you recommend a few hotels that offer complimentary breakfast? Tell me their description, address, tags, and the rate for one room that sleeps 4 people."

In [ ]:
selected_fields = ["HotelName","Description","Address","Rooms","Tags"]

search_results = search_client.search(
    search_text=query,
    top=5,
    select=selected_fields,
    query_type="semantic"
)

sources_filtered = [{field: result[field] for field in selected_fields} for result in search_results]
sources_formatted = "\n".join([json.dumps(source) for source in sources_filtered])

In [ ]:
response = openai_client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": GROUNDED_PROMPT.format(query=query, sources=sources_formatted)
        }
    ],
    model=os.getenv("AZURE_DEPLOYMENT_MODEL")
)

In [ ]:
print(response.choices[0].message.content)